# Open Targets Cancer Gene-Disease-MeSH Pipeline

This notebook demonstrates the data pipeline that maps cancer gene-disease associations to MeSH vocabulary.

**Pipeline Steps:**
1. Extract cancer diseases from Open Targets (filter by EFO_0000616)
2. Extract MeSH C04.588 hierarchy live from d2025.bin
3. Build gene-MeSH crosswalk with Entrez IDs

**Final Output:** `gene_disease_mesh_final.tsv`
- disease_mesh_id
- gene_entrez_id  
- ot_score
- evidence_count

In [3]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data")
PROCESSED_DIR = DATA_DIR / "processed"

pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 200)

---
## Final Output

The 4-column TSV for patent matching:

In [6]:
final = pd.read_csv(PROCESSED_DIR / "gene_disease_mesh_final.tsv", sep='\t')

print(f"Shape: {final.shape}")
print(f"Columns: {list(final.columns)}")
print(f"\nUnique MeSH diseases: {final['disease_mesh_id'].nunique()}")
print(f"Unique Entrez genes: {final['gene_entrez_id'].nunique()}")

final.tail(30)

Shape: (171856, 4)
Columns: ['disease_mesh_id', 'gene_entrez_id', 'ot_score', 'evidence_count']

Unique MeSH diseases: 146
Unique Entrez genes: 19275


,disease_mesh_id,gene_entrez_id,ot_score,evidence_count
171826,D002289,100506374,0.001043,1
171827,D018246,10935,0.001043,1
171828,D002289,2109,0.001040,1
171829,D018246,1740,0.001040,1
171830,D000077192,54813,0.001039,1
171831,D002289,89876,0.001038,1
171832,D002289,9315,0.001033,1
171833,D021441,55731,0.001033,1
171834,D009303,3397,0.001031,1
171835,D002289,642819,0.001030,1


---
## Step 1: Extract MeSH Hierarchy from d2025.bin

The pipeline extracts the C04.588 (Neoplasms by Site) branch directly from the raw MeSH 2025 file.

In [7]:
# Show raw MeSH file
mesh_path = DATA_DIR / "mesh" / "d2025.bin"
print(f"MeSH source: {mesh_path}")
print(f"Size: {mesh_path.stat().st_size / 1e6:.1f} MB")

# Show first record
print("\n=== Sample record ===")
with open(mesh_path, 'r') as f:
    lines = []
    for line in f:
        lines.append(line.strip())
        if len(lines) > 30:
            break
print('\n'.join(lines[:30]))

MeSH source: ../data/mesh/d2025.bin
Size: 31.4 MB

=== Sample record ===
*NEWRECORD
RECTYPE = D
MH = Calcimycin
AQ = AA AD AE AG AI AN BI BL CF CH CL CS EC HI IM IP ME PD PK PO RE SD ST TO TU UR
PRINT ENTRY = 4-Benzoxazolecarboxylic acid, 5-(methylamino)-2-((3,9,11-trimethyl-8-(1-methyl-2-oxo-2-(1H-pyrrol-2-yl)ethyl)-1,7-dioxaspiro(5.5)undec-2-yl)methyl)-, (6S-(6alpha(2S*,3S*),8beta(R*),9beta,11alpha))-|T109|T195|NON|EQV|NLM (2024)|220922|abbcdef
ENTRY = A23187|T109|T195|LAB|NRW|UNK (19XX)|741111|abbcdef
ENTRY = A-23187|T109|T195|LAB|NRW|NLM (1991)|900308|abbcdef
ENTRY = Antibiotic A23187|T109|T195|NON|NRW|NLM (1991)|900308|abbcdef
ENTRY = A 23187
ENTRY = A23187, Antibiotic
MN = D02.355.291.933.125
MN = D02.540.576.625.125
MN = D03.633.100.221.173
MN = D04.345.241.654.125
MN = D04.345.674.625.125
PA = Anti-Bacterial Agents
PA = Calcium Ionophores
MH_TH = FDA SRS (2014)
MH_TH = NLM (1975)
ST = T109
ST = T195
RN = 37H9VM9WZL
RR = 52665-69-7 (Calcimycin)
PI = Antibiotics (1973-1974)
PI = 

In [8]:
# Extract C04.588 hierarchy (this is what the pipeline does live)
import sys
sys.path.insert(0, '..')
from src.pipeline.extract_mesh import parse_mesh_file, extract_c04_hierarchy

records = parse_mesh_file(mesh_path)
print(f"Total MeSH descriptors: {len(records):,}")

# Extract C04.588 (site only)
site_hierarchy = extract_c04_hierarchy(records, prefix="C04.588")
print(f"\nC04.588 (Neoplasms by Site):")
print(f"  Tree paths: {len(site_hierarchy)}")
print(f"  Unique terms: {site_hierarchy['mesh_id'].nunique()}")
print(f"  Levels: {site_hierarchy['level'].min()}-{site_hierarchy['level'].max()}")

site_hierarchy.head(15)

Total MeSH descriptors: 30,954

C04.588 (Neoplasms by Site):
  Tree paths: 271
  Unique terms: 236
  Levels: 2-9


,mesh_id,mesh_name,tree_number,level
0,D009371,Neoplasms by Site,C04.588,2
1,D000008,Abdominal Neoplasms,C04.588.033,3
2,D010534,Peritoneal Neoplasms,C04.588.033.513,4
3,D012186,Retroperitoneal Neoplasms,C04.588.033.731,4
4,D058288,Sister Mary Joseph's Nodule,C04.588.033.740,4
5,D000694,Anal Gland Neoplasms,C04.588.083,3
6,D001859,Bone Neoplasms,C04.588.149,3
7,D050398,Adamantinoma,C04.588.149.030,4
8,D005266,Femoral Neoplasms,C04.588.149.276,4
9,D012888,Skull Neoplasms,C04.588.149.721,4


In [9]:
# Level distribution
print("=== MeSH Level Distribution ===")
print(site_hierarchy['level'].value_counts().sort_index())

print("\n=== Sample Terms by Level ===")
for level in range(2, 7):
    terms = site_hierarchy[site_hierarchy['level'] == level]['mesh_name'].head(3).tolist()
    print(f"Level {level}: {', '.join(terms)}")

=== MeSH Level Distribution ===
2     1
3    17
4    67
5    90
6    65
7    20
8    10
9     1
Name: level, dtype: int64

=== Sample Terms by Level ===
Level 2: Neoplasms by Site
Level 3: Abdominal Neoplasms, Anal Gland Neoplasms, Bone Neoplasms
Level 4: Peritoneal Neoplasms, Retroperitoneal Neoplasms, Sister Mary Joseph's Nodule
Level 5: Jaw Neoplasms, Nose Neoplasms, Orbital Neoplasms
Level 6: Mandibular Neoplasms, Maxillary Neoplasms, Palatal Neoplasms


---
## Step 2: Cancer Diseases from Open Targets

Filter diseases where `ancestors` contains EFO_0000616 (neoplasm), then extract MeSH IDs from `dbXRefs`.

In [ ]:
diseases = pd.read_parquet(PROCESSED_DIR / "intermediate" / "cancer_diseases_mesh_crosswalk.parquet")

total = len(diseases)
with_mesh = diseases['meshIds'].notna().sum()

print("=== CANCER DISEASES ===")
print(f"Total: {total:,}")
print(f"With MeSH: {with_mesh:,} ({with_mesh/total*100:.1f}%)")
print(f"Without MeSH: {total - with_mesh:,}")

diseases[diseases['meshIds'].notna()].head(10)

---
## Step 3: Disease → MeSH Crosswalk

In [ ]:
crosswalk = pd.read_csv(PROCESSED_DIR / "crosswalks" / "disease_mesh_crosswalk.csv")

print("=== DISEASE → MeSH CROSSWALK ===")
print(f"Pairs: {len(crosswalk):,}")
print(f"Diseases: {crosswalk['diseaseId'].nunique()}")
print(f"MeSH terms: {crosswalk['meshId'].nunique()}")

crosswalk.head(10)

---
## Step 4: Ensembl → Entrez Gene Mapping

In [ ]:
entrez = pd.read_csv(PROCESSED_DIR / "crosswalks" / "ensembl_entrez.csv")

print("=== ENSEMBL → ENTREZ ===")
print(f"Human gene mappings: {len(entrez):,}")
print(f"Source: NCBI gene2ensembl")

entrez.head(10)

---
## Analysis: Top Cancer Sites by Gene Count

In [ ]:
# Get MeSH names for the final data
mesh_names = site_hierarchy[['mesh_id', 'mesh_name']].drop_duplicates()

top_sites = final.groupby('disease_mesh_id').agg({
    'gene_entrez_id': 'nunique',
    'ot_score': 'mean',
    'evidence_count': 'sum'
}).reset_index()
top_sites.columns = ['mesh_id', 'unique_genes', 'mean_score', 'total_evidence']

top_sites = top_sites.merge(mesh_names, on='mesh_id', how='left')
top_sites = top_sites.sort_values('unique_genes', ascending=False)

print("=== TOP 20 CANCER SITES BY GENE COUNT ===")
top_sites[['mesh_id', 'mesh_name', 'unique_genes', 'mean_score', 'total_evidence']].head(20)

---
## Key Decisions

| Decision | Choice | Rationale |
|----------|--------|----------|
| MeSH source | d2025.bin (live extraction) | Reproducible, no stale CSV |
| MeSH branch | C04.588 (site only) | Anatomical classification, not histologic |
| Disease source | OT dbXRefs only | Curated mappings, no external crosswalks |
| Gene IDs | Entrez (from gene2ensembl) | Required for patent matching |
| Aggregation | MAX score, SUM evidence | Collapse diseases → MeSH terms |
| Coverage | 18% of OT diseases | Expected - research vs clinical vocabulary |